In [1]:
import   pandas  as  pd
import   numpy   as  np
import   seaborn as  sns
import   matplotlib.pyplot as plt ; plt.rcParams["font.family"] = "DejaVu Serif"
from     sklearn.metrics         import mean_squared_error, r2_score, mean_absolute_percentage_error, median_absolute_error
from     sklearn.model_selection import cross_val_score,GridSearchCV, train_test_split, RepeatedKFold
from     sklearn.preprocessing   import StandardScaler, MinMaxScaler
#from deepforest import CascadeForestRegressor


### some functions

In [2]:

"""
    Calculate Mean Squared Error (MSE) between true and predicted values.

    Parameters:
        -- y_true (numpy array or list): True values
        -- y_pred (numpy array or list): Predicted values

    Returns:
        -- mse (float): Mean Squared Error
    """

def mean_squared_error(y_true, y_pred):

    #convert then into umpy array if not already
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    #squared the differences
    squared_diff = (y_true - y_pred)**2
    #find the avg of squared_diff
    mse = np.mean(squared_diff)
    return mse


"""
    Calculate Root Mean Squared Error (RMSE) between true and predicted values.

    Parameters:
        -- y_true (numpy array or list): True values
        -- y_pred (numpy array or list): Predicted values

    Returns:
        -- mse (float): Mean Squared Error
    """

def root_mean_squared_error(y_true, y_pred):

    #convert then into umpy array if not already
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    #squared the differences
    squared_diff = (y_true - y_pred)**2
    #find the avg of squared_diff
    mse = np.mean(squared_diff)
    return np.sqrt(mse)



"""
    Calculate Mean Absolute Error (MAE) between true and predicted values.

    Parameters:
        -- y_true (numpy array or list): True values
        -- y_pred (numpy array or list): Predicted values

    Returns:
        -- mae (float): Mean Absolute Error
    """

def mean_absolute_error(y_true, y_pred):

    #convert then into umpy array if not already
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    #squared the differences
    absolute_diff = np.absolute(y_true - y_pred)
    #find the avg of squared_diff
    mae = np.mean(absolute_diff)
    return mae


"""
    Calculate R-squared value given predicted value and actual label

    Parameters:
        -- y_true (numpy array or list): True values
        -- y_pred (numpy array or list): Predicted values

    Returns:
        -- r^2 (float): Coefficient of determination(R^2)
    """

# def r2_score(y_true, y_pred):

#     #convert then into umpy array if not already
#     y_true = np.array(y_true)
#     y_pred = np.array(y_pred)
#     #sum of squared residuals
#     SSR = np.sum((y_true - y_pred)**2)
#     #sum of squared total
#     y_avg = np.mean(y_true)
#     SST = np.sum((y_avg-y_true)**2)
#     return 1 - (float(SSR)/float(SST))


"""
    Calculate Adjusted R-squared value given predicted and actual label and number of predictors

    Parameters:
        -- y_true (numpy array or list): True values
        -- y_pred (numpy array or list): Predicted values
        -- k (integer): Number of predictors

    Returns:
        -- adjusted r^2 (float): Coefficient of determination(R^2)
    """

def adjusted_r2_score(y_true, y_pred, k):

    #convert then into umpy array if not already
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    #number of observations
    n = len(y_true)
    #sum of squared residuals
    SSR = np.sum((y_true - y_pred)**2)
    #sum of squared total
    y_avg = np.mean(y_true)
    SST = np.sum((y_avg-y_true)**2)

    #r2_score
    r2_score = 1 - (float(SSR)/float(SST))
    #adjusted r2 square
    r2_score_adj = 1 - float((1-r2_score**2)*(n-1))/float(n - k - 1)

    return r2_score_adj

In [3]:
###  python== 3.9.13
### pandas==1.3.3 \    numpy==1.23.5  \ openpyxl==3.0.10  \  matplotlib==3.5.1
### xgboost==1.7.3 \ shap==0.45.0


# **PreProc**

## Loading data

In [ ]:
data3 = pd.read_excel('./Data.xlsx')
data2 = data3.drop(columns=['CEC','HM','T', 'SA',	'Label','η','Ncharge'], axis=1)
print(data2.shape); data2.head(2)

In [ ]:
new_columns = {'pH_solution':'PHS', '(O+N)/C':'ONC','pH_biochar':'PHB','(H-O-2N)/C':'HO2NC'}
data2 = data2.rename(columns=new_columns)
print(data2.shape); data2.head(2)

In [ ]:
## Find duplicate rows
duplicates = data2[data2.duplicated()]
## Display and drop the duplicate rows
print("number of duplicates shape:",duplicates.shape)
data_no_duplicates = data2.drop_duplicates()
print("data_no_duplicates shape:", data_no_duplicates.shape)
data_no_duplicates.head(2)

# data_no_duplicates = data2

In [ ]:
# # Get the category names corresponding to the numeric codes
# category_mapping = dict(enumerate(data_no_duplicates['HM'].astype('category').cat.categories))

# # View the mapping
# print(category_mapping)


In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# ## Apply OneHotEncoder
# encoder = OneHotEncoder(sparse_output=False)
# encoded_data = encoder.fit_transform(data_no_duplicates[['HM']])

# ## Convert the result back to a DataFrame
# encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['HM']))
# print(encoded_data[:2]); print("encoded_df shape is:", encoded_df.shape)

# ## Drop the original 'Metal_type' column
# data = data_no_duplicates.drop('HM', axis=1)

# ## Concatenate the encoded columns with the original DataFrame
# data.reset_index(drop=True, inplace=True)
# encoded_df.reset_index(drop=True, inplace=True)

# data = pd.concat([data, encoded_df], axis=1)


# ## Ensure 'RE' is the last column
# columns = [col for col in data.columns if col != 'RE'] + ['RE']
# data = data[columns]

# ## Now 'data' contains the one-hot encoded columns, and 'AE' is the last column
# data = data.drop(columns=['O/C','ONC'], axis=1)
# print("\n", data.head(2)); print("data shape is:", data.shape)

# Get rows with NaN
# rows_with_nan = data[data.isna().any(axis=1)]
# print(rows_with_nan)

In [ ]:
# Get rows with NaN
rows_with_nan = data_no_duplicates[data_no_duplicates.isna().any(axis=1)]
print(rows_with_nan)


In [ ]:
# # Get category names corresponding to the numeric codes
# categories = encoder.categories_[0]  # Since we're encoding just one column ('Metal_type')

# # Print the category names
# print("Category names:", categories)

In [ ]:
inputs = data_no_duplicates.iloc[:,:-1]
output = data_no_duplicates.iloc[:,-1]
print(inputs.shape)
inputs.head()

# Cd2+ = [0.0000	1.0000	0.0000	0.0000	0.0000	0.0000]
# Cu2+ = [0.0000	0.0000	1.0000	0.0000	0.0000	0.0000]
# Pb2+ = [0.0000	0.0000	0.0000	0.0000	1.0000	0.0000]
# Zn2+ = [0.0000	0.0000	0.0000	0.0000	0.0000	1.0000]
# As3+ = [1.0000	0.0000	0.0000	0.0000	0.0000	0.0000]
# Ni2+ = [0.0000	0.0000	0.0000	1.0000	0.0000	0.0000]


In [ ]:
output.describe()

In [ ]:
output.head(3)

In [ ]:
pd.options.display.float_format = '{:,.4f}'.format
inputs.iloc[:,:-6].describe()

In [ ]:
print(inputs.dtypes)
# print(inputs['C0'].unique())
# Get the unique values in the 'C0' column
unique_values = inputs['C0'].unique()

# Loop through the unique values and find their indices
for value in unique_values:
    indices = inputs[inputs['C0'] == value].index
    # print(f"Value: {value} - Indices: {indices.tolist()}")


In [ ]:
### corr = data_mod_lc.drop('RE (%)', axis=1).corr() # examining correlations

### Plotting correlation above or below 0.5
### df = data.drop(columns=['Zinc', 'Cadmium', 'Arsenic'], axis=1)

### new_columns = {'Siq (MPa)':'CSR', 'Hs (m)':'PLS','Hr (m)':'PLR','B (m)':'PD' }
### df = data.rename(columns=new_columns)

# corr = data.iloc[:,:-1].corr()

# plt.rcParams["font.family"] = "DejaVu Serif"

# plt.figure(figsize=(12, 10))

# sns.heatmap(corr[(corr >= 0.0) | (corr <= -0.0)],
#             cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1,
#             annot=True, annot_kws={"size": 8}, square=True);
# plt.tight_layout()
# plt.savefig('./Res/Figs/corrEL_2.pdf', dpi=600, bbox_inches='tight', format = 'pdf')
# plt.show()

## Split and Save

### IsolationForest

In [ ]:
from sklearn.ensemble import IsolationForest
import pandas as pd

## Assuming 'data_no_duplicates' is DataFrame and you want to remove rows with outliers

## Define the IsolationForest model
iso_forest = IsolationForest(random_state=42)  # Adjust contamination based on expected outlier percentage

## Fit the model and predict outliers
aaaaa = pd.DataFrame(data_no_duplicates)
aaaaa.to_excel('./TTV/data_with_outliers.xlsx', index= False)
isodata = iso_forest.fit_predict(data_no_duplicates)

## Filter out outliers (outliers will have a label of -1)
data_no_outliers = data_no_duplicates[isodata != -1]

In [ ]:
inputs = data_no_outliers.iloc[:,:-1]
output = data_no_outliers.iloc[:,-1]
print(inputs.shape)
inputs.head(3)

# aaaaa = pd.DataFrame(data_no_outliers)
# aaaaa.to_excel('./TTV/withoutOutlier/data_no_outliers.xlsx', index= False)

### Train_Test_Split

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# 1. Separate your data (as you already have)
# Note: Ensure 'Label' is dropped first if it's still in the inputs, 
# as scalers only work on numbers.

# inputs = data_no_outliers.iloc[:, :-1] # Assuming Label is the 2nd to last column
# output = data_no_outliers.iloc[:, -1]
inputs = data_no_duplicates.iloc[:,:-1]
output = data_no_duplicates.iloc[:,-1]


# 2. Initialize the scalers
scaler_in = MinMaxScaler(feature_range=(0, 1))
scaler_out = MinMaxScaler(feature_range=(0, 1))

# 3. Scale the Inputs
# This returns a numpy array. We convert back to DataFrame to keep column names.
inputs_scaled = pd.DataFrame(
    scaler_in.fit_transform(inputs), 
    columns=inputs.columns
)

# 4. Scale the Output
# Scikit-learn expects a 2D array for the output, so we use .values.reshape(-1, 1)
output_scaled = scaler_out.fit_transform(output.values.reshape(-1, 1))

# Flatten it back to a 1D array/series for the regression model
output_scaled = output_scaled.flatten()

# --- Verification ---
print(f"Input Max: {inputs_scaled.max().max()}") # Should be 1.0
print(f"Output Max: {output_scaled.max()}")      # Should be 1.0


trainx1, testx1, trainy1, testy1= train_test_split(inputs_scaled, output_scaled, test_size=0.2, random_state=22 , shuffle=True)

# trainx1, testx1, trainy1, testy1= train_test_split(inputs, output, test_size=0.2, random_state=22 , shuffle=True)

In [ ]:
trainx1

In [ ]:
# aaaaa = pd.DataFrame(trainx1)
# aaaaa.to_excel('./TTV/withoutOutlier/trainx.xlsx', index= False)
# bbbb = pd.DataFrame(trainy1)
# bbbb.to_excel('./TTV/withoutOutlier/trainy.xlsx', index= False)

# aaaaa = pd.DataFrame(testx1)
# aaaaa.to_excel('./TTV/withoutOutlier/testx.xlsx', index= False)
# bbbb = pd.DataFrame(testy1)
# bbbb.to_excel('./TTV/withoutOutlier/testy.xlsx')

trainx2 = pd.read_excel('./TTV/withoutOutlier/trainx.xlsx')
trainy2 = pd.read_excel('./TTV/withoutOutlier/trainy.xlsx')
testx2 = pd.read_excel('./TTV/withoutOutlier/testx.xlsx')
testy2 = pd.read_excel('./TTV/withoutOutlier/testy.xlsx')


# testy2 = testy2.iloc[:,1]
# trainy2 = trainy2.iloc[:,1]

# trainx2 = trainx2.iloc[:,1:]
# testx2 = testx2.iloc[:,1:]

# x_train = trainx2.to_numpy()
# x_test =  testx2.values.astype(np.float64)

# y_test  = testy2
# y_train = trainy2


In [ ]:
# aaaaa = pd.DataFrame(trainx1)
# aaaaa.to_excel('./TTV/trainx.xlsx', index= False)
# bbbb = pd.DataFrame(trainy1)
# bbbb.to_excel('./TTV/trainy.xlsx', index= False)

# aaaaa = pd.DataFrame(testx1)
# aaaaa.to_excel('./TTV/testx.xlsx', index= False)
# bbbb = pd.DataFrame(testy1)
# bbbb.to_excel('./TTV/testy.xlsx')

trainx2 = pd.read_excel('./TTV/trainx.xlsx')
trainy2 = pd.read_excel('./TTV/trainy.xlsx')
testx2 = pd.read_excel('./TTV/testx.xlsx')
testy2 = pd.read_excel('./TTV/testy.xlsx')


# testy2 = testy2.iloc[:,1]
# trainy2 = trainy2.iloc[:,1]

# trainx2 = trainx2.iloc[:,1:]
# testx2 = testx2.iloc[:,1:]

# x_train = trainx2.to_numpy()
# x_test =  testx2.values.astype(np.float64)

# y_test  = testy2
# y_train = trainy2


### GAnetwork

In [ ]:
# train a generative adversarial network on a one-dimensional function
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense, Input
from matplotlib import pyplot

# define the standalone discriminator model
def define_discriminator(n_inputs=2):
    model = Sequential()
    model.add(Input(shape=(n_inputs,)))
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(60, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(20, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator(latent_dim, n_outputs=2):
    model = Sequential()
    model.add(Input(shape=(latent_dim,)))
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(60, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(20, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(n_outputs, activation='linear'))
    return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

# generate n real samples with class labels
def generate_real_samples(n):
    # # generate inputs in [-0.5, 0.5]
    # X1 = rand(n) - 0.5
    # # generate outputs X^2
    # X2 = X1 * X1
    # # stack arrays
    # X1 = X1.reshape(n, 1)
    # X2 = X2.reshape(n, 1)
    # X = hstack((X1, X2))
    # # generate class labels
    # y = ones((n, 1))
    X = XN
    y = YN

    return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n):
    # generate points in the latent space
    x_input = randn(latent_dim * n)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n, latent_dim)
    return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n)
    # predict outputs
    X = generator.predict(x_input)
    # create class labels
    y = zeros((n, 1))
    return X, y

# evaluate the discriminator and plot real and fake points
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):
    # prepare real samples
    x_real, y_real = generate_real_samples(n)
    # evaluate discriminator on real examples
    _, acc_real = discriminator.evaluate(x_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(generator, latent_dim, n)
    # evaluate discriminator on fake examples
    _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print(epoch, acc_real, acc_fake)
    # scatter plot real and fake data points
    pyplot.scatter(x_real[:, 0], x_real[:, 1], color='red')
    pyplot.scatter(x_fake[:, 0], x_fake[:, 1], color='blue')
    pyplot.show()

# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=10000, n_batch=128, n_eval=2000):
    # determine half the size of one batch, for updating the discriminator
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # prepare real samples
        x_real, y_real = generate_real_samples(half_batch) 
        # prepare fake examples
        x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch) 
        # update discriminator
        d_model.train_on_batch(x_real, y_real, verbose=0) 
        d_model.train_on_batch(x_fake, y_fake, verbose=0) 
        # prepare points in latent space as input for the generator
        x_gan = generate_latent_points(latent_dim, n_batch) 
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 1))
        # update the generator via the discriminator's error
        gan_model.train_on_batch(x_gan, y_gan, verbose=0) 
        # evaluate the model every n_eval epochs
        # if (i+1) % n_eval == 0:
            #summarize_performance(i, g_model, d_model, latent_dim)

# size of the latent space
latent_dim = 19
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)

# **Models**

## C1 without outlier

In [ ]:
trainx2 = pd.read_excel('./TTV/withoutOutlier/trainx.xlsx')
trainy2 = pd.read_excel('./TTV/withoutOutlier/trainy.xlsx')
testx2 = pd.read_excel('./TTV/withoutOutlier/testx.xlsx')
testy2 = pd.read_excel('./TTV/withoutOutlier/testy.xlsx')
trainy2

In [ ]:
trainx = pd.read_excel('./TTV/withoutOutlier/trainx.xlsx')
trainy = pd.read_excel('./TTV/withoutOutlier/trainy.xlsx').values.ravel()
testx = pd.read_excel('./TTV/withoutOutlier/testx.xlsx')
testy = pd.read_excel('./TTV/withoutOutlier/testy.xlsx').values.ravel()
trainx

In [ ]:
trainy.shape

### KAN

In [6]:
import pandas as pd
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
from kan import KAN
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import warnings

warnings.filterwarnings("ignore")

# ---------------------------------------------------------
# 1. PATHS & LOADING
# ---------------------------------------------------------
save_path = r'./Res/kan'
os.makedirs(save_path, exist_ok=True)

def load_clean(path):
    df = pd.read_excel(path).dropna(how='all')
    return df.apply(pd.to_numeric, errors='coerce').dropna()

trainx_df = load_clean('./TTV/trainx.xlsx')
trainy_df = load_clean('./trainy.xlsx')
testx_df  = load_clean('./testx.xlsx')
testy_df  = load_clean('./TTV/testy.xlsx')

# ---------------------------------------------------------
# 2. HYBRID PREPROCESSING
# ---------------------------------------------------------
# StandardScaler for inputs (handles different units better)
scaler_x = StandardScaler()
# MinMaxScaler for output (keeps target in 0-1 range)
scaler_y = MinMaxScaler(feature_range=(0, 1))

X_train_s = scaler_x.fit_transform(trainx_df.values)
y_train_s = scaler_y.fit_transform(trainy_df.values.reshape(-1, 1))
X_test_s  = scaler_x.transform(testx_df.values)
y_test_s  = scaler_y.transform(testy_df.values.reshape(-1, 1))

dataset = {
    'train_input': torch.tensor(X_train_s, dtype=torch.float32),
    'train_label': torch.tensor(y_train_s, dtype=torch.float32),
    'test_input':  torch.tensor(X_test_s,  dtype=torch.float32),
    'test_label':  torch.tensor(y_test_s,  dtype=torch.float32)
}

d = X_train_s.shape[1]

# ---------------------------------------------------------
# 3. MODEL DEFINITION (Single Hidden Layer - High Stability)
# ---------------------------------------------------------
print(f"\n>>> Initializing KAN [{d}, 8, 1]...")
# A single wide layer (8) is often better than two thin layers
model = KAN(width=[d, 8, 1], grid=3, k=3, seed=42)

# ---------------------------------------------------------
# 4. BALANCED TRAINING STRATEGY
# ---------------------------------------------------------

# Stage 1: Warmup with very coarse grid
print(">>> Stage 1: Adam Warmup (Grid 3)")
model.fit(dataset, opt="Adam", steps=300, lamb=0.002)

# Stage 2: Increase resolution
print(">>> Stage 2: Grid Extension (Grid 10)")
model = model.refine(10)
model.fit(dataset, opt="Adam", steps=300, lamb=0.002)

# Stage 3: Fine Tuning with LBFGS
print(">>> Stage 3: LBFGS Precision Tuning (Grid 20)")
model = model.refine(20)
model.fit(dataset, opt="LBFGS", steps=100, lamb=0.001, update_grid=False)

# Stage 4: Pruning
print(">>> Stage 4: Pruning")
model = model.prune()
# Final polish after pruning
model.fit(dataset, opt="LBFGS", steps=50, update_grid=False)

# ---------------------------------------------------------
# 5. EVALUATION
# ---------------------------------------------------------
y_train_pred_s = model(dataset['train_input']).detach().numpy()
y_test_pred_s  = model(dataset['test_input']).detach().numpy()

y_train_pred = scaler_y.inverse_transform(y_train_pred_s).ravel()
y_test_pred  = scaler_y.inverse_transform(y_test_pred_s).ravel()
y_train_real = scaler_y.inverse_transform(y_train_s).ravel()
y_test_real  = scaler_y.inverse_transform(y_test_s).ravel()

# Clip predictions to 0-1 range to handle spline overshoots
y_train_pred = np.clip(y_train_pred, 0, 1)
y_test_pred = np.clip(y_test_pred, 0, 1)

r2_train = r2_score(y_train_real, y_train_pred)
r2_test  = r2_score(y_test_real,  y_test_pred)

print(f"\n================ FINAL RESULTS ================")
print(f"Train R² : {r2_train:.4f}")
print(f"Test  R² : {r2_test:.4f}")

# ---------------------------------------------------------
# 6. PLOTTING
# ---------------------------------------------------------
plt.figure(figsize=(7, 6))
plt.scatter(y_train_real, y_train_pred, alpha=0.3, label=f'Train (R²={r2_train:.2f})', color='royalblue')
plt.scatter(y_test_real, y_test_pred, alpha=0.7, edgecolors='k', label=f'Test (R²={r2_test:.2f})', color='darkorange')
plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Ideal')
plt.xlabel('Measured')
plt.ylabel('Predicted')
plt.title(f'Balanced PyKAN (Test R²: {r2_test:.3f})')
plt.legend()
plt.tight_layout()
plt.show()

# 7. Symbolic Formula
try:
    model.auto_symbolic(lib=['x', 'x^2', 'exp', 'sin'], verbose=False)
    formula = model.symbolic_formula()[0][0]
    print("\nExtracted Equation:\n", formula)
except:
    print("\nVisualizing via model.plot()...")
    model.plot()


ImportError: cannot import name 'KAN' from 'kan' (/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kan/__init__.py)

### pysr

### pysindy

### RILS

### gplearn

In [ ]:
# import sklearn
# print("scikit-learn version:", sklearn.__version__)
# scikit-learn version: 1.7.2